In [1]:
import tensorflow as tf
import pandas as pd
import re
train = pd.read_csv("./training_data.csv", header=0, delimiter="\t|\n")
data = train["comment"]
### Text Preprocessing
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
keep_stopwords = stopwords.words("english")[-36:]+stopwords.words("english")[131:133] + stopwords.words("english")[116:120]
new_stopwords = set(stopwords.words("english")).difference(set(keep_stopwords))
replace_num = "[-+]?[0-9]*\.?[0-9]+"
replace_url = "http(s?)://[^\s]+"
def review_to_words(raw_review):
    
    review_text = BeautifulSoup(raw_review).get_text()
    review_text = re.sub(r'[\t\r\n]'," ",review_text)
    url = re.sub(replace_url,"URL",raw_review)
    num = re.sub(replace_num,"NUM",url)
    letters_only = re.sub("[^a-zA-Z']", " ", num) #Also keep single quote
    
    words = letters_only.lower().split()
    
    meaningful_words = [w for w in words if not w in new_stopwords]
    
    return(" ".join(meaningful_words))

clean_data = dict()    
clean_data["x"] = []
clean_data["y"] = []
import bar
step=0
total = len(data)
for each in data:
    clean_data["x"].append(review_to_words(each))
    bar.drawProgressBar(step/total)
    step +=1




/home/kevindong1994/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/kevindong1994/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  after removing the cwd from sys.path.


[                      ] 0.12%

/home/kevindong1994/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /home/kevindong1994/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


[ =============        ] 65.92%

/home/kevindong1994/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


[ ===================  ] 100.00%

In [2]:
clean_data["y"] = train["score"]
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(clean_data["x"], clean_data["y"], test_size=0.33, random_state=42)

0          0
1          7
2         10
3          7
4          6
5          3
6          7
7          7
8          7
9          6
10         5
11         1
12         8
13         7
14         5
15         8
16        10
17         7
18        10
19         8
20        10
21        10
22        10
23         8
24         8
25         8
26         5
27        10
28         9
29         0
          ..
151794     3
151795     7
151796     3
151797     3
151798     5
151799     1
151800     3
151801     1
151802    10
151803     8
151804    10
151805     9
151806     9
151807    10
151808     6
151809     4
151810    10
151811    10
151812     7
151813    10
151814    10
151815    10
151816    10
151817    10
151818    10
151819    10
151820    10
151821    10
151822     9
151823    10
Name: score, Length: 151824, dtype: int64

In [3]:
from sklearn.feature_extraction.text import CountVectorizer  
num_features = 500
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = num_features) 
train_data_features = vectorizer.fit_transform(X_train)
train_data_features = train_data_features.toarray()


In [ ]:
batch_size = 30
epochs = 50
epoch = 0
ids = list(range(0,train_data_features.shape[0],batch_size))
i=0
def generate_batch(features,labels):
    global i
    global ids
    global epoch
    if i==len(ids)-1:
        inner_i = i
        i = 0
        epoch += 1
        return features[ids[inner_i]:],labels[ids[inner_i]:]
    else:
        inner_i = i
        i +=1
        return features[ids[inner_i]:ids[i]],labels[ids[inner_i]:ids[i]]
    


In [ ]:

H = 300
with graph.as_default():
    x = tf.placeholder(tf.float32,shape=[None,num_features])
    y_ = tf.placeholder(tf.float32,shape=[None,1])
    fc_l1 = tf.layers.dense(x,units=H,activation=tf.nn.relu, kernel_initializer=tf.random_normal_initializer(),\
                     kernel_regularizer=tf.contrib.layers.l2_regularizer(0.0))
    output = tf.layers.dense(fc_l1,units=1,kernel_initializer=tf.random_normal_initializer())
    
    loss = tf.losses.mean_squared_error(output,new_y)
    opt = tf.train.AdamOptimizer(learning_rate=0.01)
    opt_op = opt.minimize(loss)
    init = tf.global_variables_initializer()
 

In [ ]:
# with tf.Session(graph = graph) as session:

session = tf.Session(graph = graph)
session.run(init)
step = 0 
print("Initialized")
while epoch<epochs:
    step+=1
    batch_x,batch_y = generate_batch(train_data,y_train)
    feed_dict={x:batch_x,y_:batch_y}
    _,loss_val = session.run([opt_op,loss],feed_dict=feed_dict)
    if step % 2000 == 0:
        print("Step: %i, Epoch: %i, Loss:%f"%(step,epoch,loss_val))
    
